In [43]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from imblearn.over_sampling import SMOTE

# Load the dataset
df = pd.read_csv("Iowa_Executive_Branch_Voluntary_and_Involuntary_Employment_Separations.csv")
df.head()

,Record Number,Fiscal Year,Department Name,Sub-Unit of Department,EEO Category Name,Reason,Employee Status,Pay Plan,Pay Grade,Classification Title,Separation Date,Pay Period End Date,Current Fiscal Year
0,12647,2013,"Administrative Services, Department of",NaN,Skilled Craft,Involuntary Dismissal,Permanent (Merit),014,21+13.5%,PAINTER 2,06/22/2012,07/05/2012,False
1,12648,2013,"Commerce, Department of",Insurance Division,Professional,Retirement,Permanent (Merit),014,29,COMPLIANCE OFFICER 1,06/22/2012,07/05/2012,False
2,12649,2013,"Corrections, Department of",Oakdale,Protective Services - Sworn,Retirement,Permanent (Merit),014,23,CORRECTIONAL OFFICER,06/22/2012,07/05/2012,False
3,12650,2013,"Education, Department of",Vocational Rehabilitation,Official/Administrator,Retirement,Permanent (Merit),000,29,EXEC OFF 1,06/22/2012,07/05/2012,False
4,12651,2013,Iowa Workforce Development,NaN,Professional,Retirement,Permanent (Merit),014,30,MANAGEMENT ANALYST 3,06/22/2012,07/05/2012,False


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14499 entries, 0 to 14498
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Record Number           14499 non-null  int64 
 1   Fiscal Year             14499 non-null  int64 
 2   Department Name         14499 non-null  object
 3   Sub-Unit of Department  8312 non-null   object
 4   EEO Category Name       14454 non-null  object
 5   Reason                  14499 non-null  object
 6   Employee Status         14499 non-null  object
 7   Pay Plan                14454 non-null  object
 8   Pay Grade               14454 non-null  object
 9   Classification Title    14454 non-null  object
 10  Separation Date         14499 non-null  object
 11  Pay Period End Date     14499 non-null  object
 12  Current Fiscal Year     14499 non-null  bool  
dtypes: bool(1), int64(2), object(10)
memory usage: 1.3+ MB


In [45]:
df.describe()

,Record Number,Fiscal Year
count,14499.000000,14499.000000
mean,20596.715842,2017.352438
std,4592.388755,2.602261
min,12647.000000,2013.000000
25%,16627.500000,2015.000000
50%,20591.000000,2018.000000
75%,24563.500000,2019.000000
max,28580.000000,2022.000000


In [46]:
# Data Cleaning: Remove duplicates
df.drop_duplicates(inplace=True)

# Convert dates to datetime format
df['Separation Date'] = pd.to_datetime(df['Separation Date'], errors='coerce')
df['Pay Period End Date'] = pd.to_datetime(df['Pay Period End Date'], errors='coerce')
df.head()

,Record Number,Fiscal Year,Department Name,Sub-Unit of Department,EEO Category Name,Reason,Employee Status,Pay Plan,Pay Grade,Classification Title,Separation Date,Pay Period End Date,Current Fiscal Year
0,12647,2013,"Administrative Services, Department of",NaN,Skilled Craft,Involuntary Dismissal,Permanent (Merit),014,21+13.5%,PAINTER 2,2012-06-22,2012-07-05,False
1,12648,2013,"Commerce, Department of",Insurance Division,Professional,Retirement,Permanent (Merit),014,29,COMPLIANCE OFFICER 1,2012-06-22,2012-07-05,False
2,12649,2013,"Corrections, Department of",Oakdale,Protective Services - Sworn,Retirement,Permanent (Merit),014,23,CORRECTIONAL OFFICER,2012-06-22,2012-07-05,False
3,12650,2013,"Education, Department of",Vocational Rehabilitation,Official/Administrator,Retirement,Permanent (Merit),000,29,EXEC OFF 1,2012-06-22,2012-07-05,False
4,12651,2013,Iowa Workforce Development,NaN,Professional,Retirement,Permanent (Merit),014,30,MANAGEMENT ANALYST 3,2012-06-22,2012-07-05,False


In [47]:
# Handle missing values
# Convert 'Pay Grade' to numeric, handling non-numeric values
df['Pay Grade'] = pd.to_numeric(df['Pay Grade'], errors='coerce')  # Convert to numeric, invalid values become NaN

num_imputer = SimpleImputer(strategy='median')
df[['Pay Grade']] = num_imputer.fit_transform(df[['Pay Grade']])

cat_imputer = SimpleImputer(strategy='most_frequent')
df[['Reason', 'Employee Status', 'Sub-Unit of Department']] = cat_imputer.fit_transform(df[['Reason', 'Employee Status', 'Sub-Unit of Department']])


In [48]:
# Feature Engineering
df['Separation Year'] = df['Separation Date'].dt.year
df['Recent Layoff'] = df['Separation Year'] >= 2018
df['Voluntary/Involuntary'] = df['Reason'].apply(lambda x: 'Voluntary' if 'Retirement' in x else 'Involuntary')
df.head()

,Record Number,Fiscal Year,Department Name,Sub-Unit of Department,EEO Category Name,Reason,Employee Status,Pay Plan,Pay Grade,Classification Title,Separation Date,Pay Period End Date,Current Fiscal Year,Separation Year,Recent Layoff,Voluntary/Involuntary
0,12647,2013,"Administrative Services, Department of",Field Operations,Skilled Craft,Involuntary Dismissal,Permanent (Merit),014,24.0,PAINTER 2,2012-06-22,2012-07-05,False,2012,False,Involuntary
1,12648,2013,"Commerce, Department of",Insurance Division,Professional,Retirement,Permanent (Merit),014,29.0,COMPLIANCE OFFICER 1,2012-06-22,2012-07-05,False,2012,False,Voluntary
2,12649,2013,"Corrections, Department of",Oakdale,Protective Services - Sworn,Retirement,Permanent (Merit),014,23.0,CORRECTIONAL OFFICER,2012-06-22,2012-07-05,False,2012,False,Voluntary
3,12650,2013,"Education, Department of",Vocational Rehabilitation,Official/Administrator,Retirement,Permanent (Merit),000,29.0,EXEC OFF 1,2012-06-22,2012-07-05,False,2012,False,Voluntary
4,12651,2013,Iowa Workforce Development,Field Operations,Professional,Retirement,Permanent (Merit),014,30.0,MANAGEMENT ANALYST 3,2012-06-22,2012-07-05,False,2012,False,Voluntary


In [49]:
# Encode categorical features
label_encoder = LabelEncoder()
df['Employee Status Encoded'] = label_encoder.fit_transform(df['Employee Status'])

one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_columns = one_hot_encoder.fit_transform(df[['EEO Category Name']])
encoded_df = pd.DataFrame(encoded_columns, columns=[f"EEO_{col}" for col in one_hot_encoder.get_feature_names_out()])

# Reset index before merging
df = df.reset_index(drop=True)
encoded_df = encoded_df.reset_index(drop=True)

# Drop original categorical column and merge encoded DataFrame
df = df.drop(columns=['EEO Category Name']).join(encoded_df)

# Handle class imbalance using SMOTE
X = df.drop(columns=['Reason'])
y = df['Reason']

# Convert ALL object type columns to numerical using OneHotEncoding
for column in X.select_dtypes(include=['object']).columns:
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_data = encoder.fit_transform(X[[column]])
    encoded_feature_names = encoder.get_feature_names_out([column])
    encoded_df = pd.DataFrame(encoded_data, columns=encoded_feature_names, index=X.index)
    X = X.drop(columns=[column]).join(encoded_df)

# Check class distribution before SMOTE
print("Class distribution before SMOTE:", Counter(y))

# Apply SMOTE only if there are enough samples
if len(Counter(y)) > 1 and min(Counter(y).values()) >= 2:  # Ensure enough samples for resampling
    smote = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=2)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    df_resampled = pd.concat([
        pd.DataFrame(X_resampled, columns=X.columns),
        pd.DataFrame(y_resampled, columns=['Reason'])
    ], axis=1)
    print("SMOTE applied successfully.")
else:
    print("Not enough class variation for SMOTE. Proceeding without resampling.")
    df_resampled = df

# Save the processed data
df_resampled.to_csv("processed_layoff_data.csv", index=False)
print("Preprocessing complete. Data saved as 'processed_layoff_data.csv'")


Class distribution before SMOTE: Counter({'Voluntary Resignation': 6272, 'Retirement': 5101, 'Involuntary Dismissal': 1773, 'Involuntary Layoff': 441, 'Disability/Medical': 288, 'Abandoned Job': 200, 'Involuntary Resignation': 186, 'Death': 180, 'Return to Recall': 37, 'Term Expiration': 18, 'Mandatory Retirement': 2, 'Pending Appeal': 1})
Not enough class variation for SMOTE. Proceeding without resampling.
Preprocessing complete. Data saved as 'processed_layoff_data.csv'
